In [ ]:
# !pip install scapy
# !pip install numpy

In [ ]:
import struct
import random
import numpy as np
from scapy.all import *

In [ ]:
src_addr = 'localhost'  # Адрес отправителя
dst_addr = '1.2.3.4'    # Адрес получателя

rows = 20  # Количество строк матрицы
cols = 20  # Количество столбцов матрицы

num_interactions = 2000  # Количество запросов
max_num_registers = 25   # Максимальное количество регистров в каждом пакете

In [ ]:
def generate_engine_matrix(rows, cols):
    # Создаем матрицу со случайными значениями от 1 до 100
    matrix = np.random.randint(1, 100, size=(rows, cols))
    
    # Добавляем заголовки для столбцов (C0 до C19)
    column_headers = [f"R{i}" for i in range(cols)]
    
    # Добавляем заголовки для строк (R0 до R19)
    row_headers = [f"C{i}" for i in range(rows)]
    
    # Выводим матрицу с заголовками
    print("\t".join([""] + column_headers))
    for i in range(rows):
        print("\t".join([row_headers[i]] + list(map(str, matrix[i]))))

    return matrix

In [ ]:
def create_modbus_tcp_packet(unit_id, function_code, data):
    # Преобразование заголовка Modbus/TCP
    protocol_id = 0
    length = len(data) + 2

    # Создание заголовка
    header = struct.pack('>HHHBB', transaction_id, protocol_id, length, unit_id, function_code)

    # Добавление данных
    packet = header + data

    return packet

In [ ]:
def create_modbus_response(unit_id, function_code, data):
    return create_modbus_tcp_packet(unit_id, function_code, data)

In [ ]:
def create_modbus_request(unit_id, starting_address, quantity):
    data = struct.pack('>HH', starting_address, quantity)
    modbus_request = create_modbus_tcp_packet(unit_id, 3, data)
    return modbus_request

In [ ]:
def create_modbus_response_from_matrix(matrix, starting_address, quantity):
    # Преобразование данных из матрицы в формат Modbus Response
    response_data = b''
    for value in matrix.flatten()[starting_address:starting_address + quantity]:
        response_data += struct.pack('>H', value)
    
    byte_count = len(response_data)
    return struct.pack('>B', byte_count) + response_data

In [ ]:
# Пример использования для создания Modbus запроса и записи в PCAP файл:
unit_id = 1

# Создание случайной матрицы
matrix = generate_engine_matrix(rows, cols)

packets = []  # Список для хранения пакетов
used_transaction_ids = set()  # Множество для отслеживания использованных transaction_id

In [ ]:
for _ in range(num_interactions):
    # Генерация случайных значений starting_address и quantity
    starting_address = random.randint(0, (rows * cols - 1))
    quantity = random.randint(1, max_num_registers)

    # Генерация уникального transaction_id
    while True:
        transaction_id = random.randint(0, num_interactions)
        if transaction_id not in used_transaction_ids:
            used_transaction_ids.add(transaction_id)
            break

    # Генерация случайного порта
    random_port = random.randint(1024, 65535)

    # Создание Modbus запроса
    modbus_request = create_modbus_request(unit_id, starting_address, quantity)

    # Преобразование в IP/TCP пакет с добавлением случайного порта назначения
    ip_packet_request = IP(src=src_addr, dst=dst_addr) / TCP(dport=502, sport=random_port) / modbus_request

    # Добавление в список
    packets.append(ip_packet_request)

    # Обработка запроса и создание Modbus Response
    modbus_response_data = create_modbus_response_from_matrix(matrix, starting_address, quantity)
    modbus_response = create_modbus_response(unit_id, 3, modbus_response_data)

    # Преобразование в IP/TCP пакет с добавлением случайного порта назначения
    ip_packet_response = IP(src=dst_addr, dst=src_addr) / TCP(dport=random_port, sport=502) / modbus_response

    # Добавление в список
    packets.append(ip_packet_response)

In [ ]:
# Запись в PCAP файл
pcap_filename = 'modbus_interactions.pcap'

wrpcap(pcap_filename, packets)
pcap_p = rdpcap(pcap_filename)
pcap_p[0]